# Homework: Agents

In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

## Preparation
First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [10]:
import random

# our fake database
known_weather_data = {
    'berlin': 20.0
}


def get_weather(city: str) -> float:
    """Generates fake weather data (temperature)"""
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [11]:
get_weather("Dnipro")

10.9

## Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts

```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```
What did you put in TODO3?

In [12]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Use weather tool to get temperature for a city."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

**Answer**: `city` (name of the function argument)

## Testing it (Optional)

If you have OpenAI API Key (or alternative provider), let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers from the system.

You can use `chat_assistant.py` or implement everything yourself

In [10]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py"
urllib.request.urlretrieve(url, "chat_assistant.py")

('chat_assistant.py', <http.client.HTTPMessage at 0x1d32105cb80>)

In [13]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather for a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Use weather tool to get temperature for a city.'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [14]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [15]:
developer_prompt = """
You're a weather application.
You're to return a response with temperature in Celsius given a city.

Use weather tool to get a temperature.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [16]:
chat.run()

Chat ended.


# Q2. Adding another tool
Let's add another tool - a function that can add weather data to our database:

```python
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [20]:
def set_weather(city: str, temp: float) -> None:
    """Add weather data to database"""
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# description
add_weather_description = {
    "type": "function",
    "name": "set_weather",
    "description": "Add weather data to our database for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City of the temperature to be added to our database."
            },
            "temp": {
                "type": "number",
                "description": "Temperature in Celcius to be added to our database."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [21]:
# adding new tool
tools.add_tool(set_weather, add_weather_description)

# tune system prompt to include new action
developer_prompt = """
You're a weather application.
You're to return a response with temperature in Celsius given a city.

For each city request:
1. First use get_weather tool to get the temperature for the city
2. Then use set_weather tool to save this temperature data to the database
3. Respond with the temperature information

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather for a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Use weather tool to get temperature for a city.'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Add weather data to our database for a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City of the temperature to be added to our database.'},
    'temp': {'type': 'number',
     'description': 'Temperature in Celcius to be added to our database.'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [22]:
chat.run()

Chat ended.


In [ ]:
# our fake database
known_weather_data

{'berlin': 20.0, 'dnipro': 34.7, 'new york': -1.3}

## MCP
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server
## Q3. Install FastMCP
Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):
```python
pip install fastmcp
```
What's the version of FastMCP you installed?

In [ ]:
%pip install fastmcp

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------------------------------ --------- 7.3/9.6 MB 45.4 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 39.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 66.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/587.4 kB ? eta -:--:--
   ---------------------------------------- 587.4/587.4 kB ? eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11117 sha256=73ed1ab9f6f6947761cc1de9e3fc0243e74423bc7ff820f25a8f49e89e57141c
  Stored in directory: c:\users\chetv\appdata\local\pip\cache\wheels\cc\ae\36\ee17d1de094fcb61e24106cb329b5103861e819f94bef5e10a
Successfully built pyperclip

  Attempting uninstall: pywin32

    Found existing installation: pywi

  DEPRECATION: Building 'pyperclip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyperclip'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [25]:
import fastmcp

fastmcp.__version__

'2.10.5'

A simple MCP server from the documentation looks like that:
```python
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:
```python
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```
Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:
```python
Starting MCP server 'Demo 🚀' with transport '<TODO>'
```
What do you have instead of <TODO>?

**Answer**: `stdio`


```
(llm_zoomcamp) PS C:\Users\chetv\Documents\llm-zoomcamp\workshops\agents> fastmcp run weather_server.py


╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                           │
│    📦 Transport:       STDIO                                               │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                             │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯


[07/14/25 11:44:40] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio'                                server.py:1371
```

## Q5. Protocol
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

First, we send an initialization request -- this way, we register our client with the server:
```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```
We should get back something like that, which is an aknowledgement of the request:
```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```
Next, we reply back, confirming the initialization:
```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```
We don't expect to get anything in response
Now we can ask for a list of available methods:
```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```
Let's ask the temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
```
What did you get in response?

### MCP server `STDIO` logs

1) initialization:
```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```
2) response:
```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.11.0"}}}
```
3) confirm initialization:
```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```
4) Methods:
```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```
5) Response:
```json
{"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true}},{"name":"set_weather","description":"Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.","inputSchema":{"properties":{"city":{"title":"City","type":"string"},"temp":{"title":"Temp","type":"number"}},"required":["city","temp"],"type":"object"}}]}}
```
6) Query for temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"Berlin"}}}
```
7) Response (Q5 answer):
```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```

## Q6. Client
We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

FastMCP also supports MCP clients:
```python
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```
Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:
```python
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```
If you run it in a script, you will need to use asyncio:
```python
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when filling in the homework form.

In [41]:
from fastmcp import Client
import weather_server

async with Client(weather_server.mcp) as mcp_client:
    available_tools = await mcp_client.list_tools()

In [42]:
available_tools

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None),
 Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'T

## Using tools from the MCP server (optional)
FastMCP uses `asyncio` for client-server communication. In our case, the code we wrote previously in the module (`chat_assistant.py`) is not asyncio-friendly, so it will require a lot of adjustments to run it.

Which is why we asked Claude to implement a simple non-async MCP client (see `mcp_client.py`) that can only do this:

* List tools
* Invoke the specified tool
Note: this is not a production-ready MCP Client! Use it only for learning purposes.

Check the code - it's quite illustrative. Or experiment with writing this code yourself.

Here's how we can use it:
```python
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()
```

While it's somewhat verbose, it follows the initialization structure we outlined in Q5.

Now we can use it:
```python
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})
```

In order to include it in our existing application, we need a wrapper class:
```python
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

```
It's very similar to the Tools class we created in the module, but it uses MCP to communicate with the MCP Server.

(Where convert_tools_list converts MCP functions description format into the OpenAI's one)

Let's use it:
```python
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()
```

Now we use the MCP server for function calling!

In [44]:
import urllib.request

url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/cohorts/2025/0a-agents/mcp_client.py"
urllib.request.urlretrieve(url, "mcp_client.py")

('mcp_client.py', <http.client.HTTPMessage at 0x25886f9a500>)

In [45]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [46]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [47]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Dnipro'}


Chat ended.
